In [1]:
import matplotlib.pyplot as plt
import nest
import numpy as np

import os
import sys
    
from pynestml.frontend.pynestml_frontend import generate_nest_target
NEST_SIMULATOR_INSTALL_LOCATION = nest.ll_api.sli_func("statusdict/prefix ::")   
    
%matplotlib inline

<frozen importlib._bootstrap>:241: RuntimeWarning: compiletime version 3.8 of module 'pynestkernel' does not match runtime version 3.1
<frozen importlib._bootstrap>:241: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 880 from C header, got 888 from PyObject



              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: master@61f08e0ea
 Built: Jul 14 2022 17:28:07

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [2]:
co_creation = 0
exc_edlif = 1
inh_edlif = 0

nest.ResetKernel()

if co_creation:
    # neuron and synapse co-created
    nest.Install("nestml_0_module")
    #exc_neuron_model_name = "edlif_percent_mwe0_nestml__with_stdp0_nestml" # "iaf_psc_alpha" "edlif_psc_percent"
    #inh_neuron_model_name = "edlif_percent_mwe0_nestml__with_stdp0_nestml"
    exc_neuron_model_name = "edlif_psc_alpha_percent0_nestml__with_stdp0_nestml"
    inh_neuron_model_name = "edlif_psc_alpha_percent0_nestml__with_stdp0_nestml"
    #synapses_model_name = "stdp0_nestml__with_edlif_percent_mwe0_nestml" # "stdp_synapse"
    synapses_model_name = "stdp0_nestml__with_edlif_psc_alpha_percent0_nestml"
    #synapses_model_name = "stdp_synapse"


else:
    # only neuron creates
    nest.Install("nestmlmodule")
    if exc_edlif:
        exc_neuron_model_name = "edlif_psc_alpha_percent" # "iaf_psc_alpha" "edlif_psc_percent"
    else:
        exc_neuron_model_name = "iaf_psc_alpha"
    if inh_edlif:
        inh_neuron_model_name = "edlif_psc_alpha_percent" # "iaf_psc_alpha" "edlif_psc_percent"
    else:
        inh_neuron_model_name = "iaf_psc_alpha"
    synapses_model_name = "stdp_synapse"





Sep 05 16:11:37 Install [Info]: 
    loaded module nestmlmodule


In [3]:
# parameters
## simulation
simtime = 100
iterations = 1
seed = 4
dim = 2
exc_circular_conn = True
inh_circular_conn = True
if dim == 2:
    mask = "circular"
elif dim == 3:
    mask = "spherical"

## excitatory neurons
exc_neuron_type = exc_neuron_model_name
exc_positions = nest.spatial.free(
    nest.random.uniform(min=-0.5, max=0.5),  # using random positions in a uniform distribution
    num_dimensions=dim  # have to specify number of dimensions
    )
exc_n = 200
exc_ndict = {"I_e": 200.0, "tau_m": 20.0}

## inhibitory neurons
inh_neuron_type = inh_neuron_model_name
inh_positions = nest.spatial.free(
    nest.random.uniform(min=-0.5, max=0.5),  
    num_dimensions=dim)
inh_n = 100
inh_ndict = {"I_e": 200.0, "tau_m": 20.0}

## noise generatos
exc_noise_type = "poisson_generator"
inh_noise_type = "poisson_generator"
exc_rate = 80000  # rate in Hz
inh_rate  = 15000

## connections
# https://nest-simulator.readthedocs.io/en/v3.3/tutorials/pynest_tutorial/part_4_spatially_structured_networks.html#pynest-tutorial-4
# https://nest-simulator.readthedocs.io/en/v3.3/guides/spatially_structured_networks.html#spatial-networks
### exc to inh
exc_d = 1.0
exc_synapse_model = synapses_model_name
print("1")
exc_inh_wr = nest.Create('weight_recorder')
nest.CopyModel(exc_synapse_model, "exc_synapse_rec", {"weight_recorder": exc_inh_wr})
exc_alpha_min = 0.1
exc_alpha_max = 2.
exc_alpha = nest.random.uniform(min=exc_alpha_min, max=exc_alpha_max)
exc_w_min = 0.5
exc_w_max = 5.
exc_w = nest.random.uniform(min=exc_w_min, max=exc_w_max)
Ke = 25
if exc_circular_conn:
    # Circular mask, distance-dependent connection probability with gaussian distribution
    print("exc: circular connection")
    exc_conn_dict = {'rule': 'pairwise_bernoulli',
             'p': nest.spatial_distributions.gaussian(nest.spatial.distance, std=0.2),
             'mask': {mask: {'radius': 0.35}},
             'allow_autapses': False,
             #'weight': exc_w
             }
else:
    print("exc: indegree connection")
    exc_conn_dict = {"rule": "fixed_indegree", "indegree": Ke}
    
exc_syn_dict = {"delay": exc_d, "weight": exc_w}
if not(exc_synapse_model==""):
    exc_syn_dict["synapse_model"] = "exc_synapse_rec"
    if not(exc_circular_conn):
        exc_syn_dict["alpha"] = exc_alpha

### inh to exc
inh_d = 1.0
# delays could be space-dependent
inh_synapse_model = synapses_model_name
print("5")
inh_exc_wr = nest.Create('weight_recorder')
nest.CopyModel(inh_synapse_model, "inh_synapse_rec", {"weight_recorder": inh_exc_wr})
inh_alpha_min = 0.1
inh_alpha_max = 2.
inh_alpha = nest.random.uniform(min=inh_alpha_min, max=inh_alpha_max)
# something weird here. inh_w should be negative
inh_w_min = 0.5
inh_w_max = 5.
#inh_w_min = -1.
#inh_w_max = -0.5
inh_w = nest.random.uniform(min=inh_w_min, max=inh_w_max)
# weights could be space-dependen
Ki = 20
if inh_circular_conn:
    print("inh: circular connection")
    inh_conn_dict = {'rule': 'pairwise_bernoulli',
         'p': nest.spatial_distributions.gaussian(nest.spatial.distance, std=0.2),
         'mask': {mask: {'radius': 0.5}},
         'allow_autapses': False,
         #'weight': inh_w
         }
else:
    print("inh: indegree connection")
    inh_conn_dict = {"rule": "fixed_indegree", "indegree": Ki}

inh_syn_dict = {"delay": inh_d, "weight": inh_w}
if not(inh_synapse_model==""):
    inh_syn_dict["synapse_model"] = "inh_synapse_rec"
    if not(inh_circular_conn):
        inh_syn_dict["alpha"] = inh_alpha

### noise to {exc, inh}
noise_exc_exc_w = 1.2
noise_exc_inh_w = 1.5
noise_inh_exc_w = -2.1
noise_inh_inh_w = -2.0
syn_dict_noise_exc_exc = {"weight": noise_exc_exc_w}
syn_dict_noise_exc_inh = {"weight": noise_exc_inh_w}
syn_dict_noise_inh_exc = {"weight": noise_inh_exc_w}
syn_dict_noise_inh_inh = {"weight": noise_inh_inh_w}
print("10")

1
exc: circular connection
5
inh: circular connection
10


In [4]:
# create network
nest.rng_seed = seed
## excitatory neurons
exc_pop = nest.Create(exc_neuron_type, n=exc_n, positions=exc_positions, params=exc_ndict)

## inhibitory neurons
inh_pop = nest.Create(inh_neuron_type, n=inh_n, positions=inh_positions, params=inh_ndict)

## noise
exc_noise = nest.Create(exc_noise_type)
inh_noise = nest.Create(inh_noise_type)
exc_noise.set(rate=exc_rate)  
inh_noise.set(rate=inh_rate)

## connections
### exc to inh
nest.Connect(exc_pop, inh_pop, 
             conn_spec=exc_conn_dict), 
             #syn_spec=exc_syn_dict)

### inh to exc
nest.Connect(inh_pop, exc_pop)
#             conn_spec=inh_conn_dict, 
#             syn_spec=inh_syn_dict)

### noise_{exc, inh} to {exc, inh}_pop
nest.Connect(exc_noise, exc_pop, syn_spec=syn_dict_noise_exc_exc)
nest.Connect(exc_noise, inh_pop, syn_spec=syn_dict_noise_exc_inh)
nest.Connect(inh_noise, exc_pop, syn_spec=syn_dict_noise_inh_exc)
nest.Connect(inh_noise, inh_pop, syn_spec=syn_dict_noise_inh_inh)

NESTErrors.UnknownModelName: UnknownModelName in SLI function CreateLayer_D_D: /edlif_psc_alpha_percent is not a known model name.

In [ ]:
exc_fig = nest.PlotLayer(exc_pop, nodesize=80)

In [ ]:
# exc pop (blue), inh pop (red)
if exc_circular_conn:
    exc_fig = nest.PlotLayer(exc_pop, nodesize=80)
    ctr = nest.FindCenterElement(exc_pop)
    final_fig = nest.PlotTargets(ctr, inh_pop, fig=exc_fig,
                     mask=exc_conn_dict['mask'], probability_parameter=exc_conn_dict['p'],
                     src_size=250, tgt_color='red', tgt_size=20, mask_color='red',
                     probability_cmap='Greens')

In [ ]:
inh_fig = nest.PlotLayer(inh_pop, nodesize=80)

In [ ]:
# inh pop (blue), exc pop (red)
if inh_circular_conn:
    inh_fig = nest.PlotLayer(inh_pop, nodesize=80)
    ctr = nest.FindCenterElement(inh_pop)
    final_fig = nest.PlotTargets(ctr, exc_pop, fig=inh_fig,
                     mask=inh_conn_dict['mask'], probability_parameter=inh_conn_dict['p'],
                     src_size=250, tgt_color='red', tgt_size=20, mask_color='red',
                     probability_cmap='Greens')

In [ ]:
# recording

## voltmeter
exc_voltmeter = nest.Create("voltmeter")
inh_voltmeter = nest.Create("voltmeter")
if exc_neuron_model_name in ["edlif_psc_percent", "edlif_percent_mwe0_nestml__with_stdp0_nestml"]:
    exc_voltmeter.set({"record_from": ["V_m", "ATP", "ATP_c", "ATP_s", "E_ap_tr", "E_ap_c"]})
    exc_voltmeter.set({"record_from": ["V_m"]})
else:
    exc_voltmeter.set({"record_from": ["V_m"]})
if inh_neuron_model_name in ["edlif_psc_percent", "edlif_percent_mwe0_nestml__with_stdp0_nestml"]:
    inh_voltmeter.set({"record_from": ["V_m", "ATP", "ATP_c", "ATP_s", "E_ap_tr", "E_ap_c"]})
    inh_voltmeter.set({"record_from": ["V_m"]})
else:
    inh_voltmeter.set({"record_from": ["V_m"]})
nest.Connect(exc_voltmeter, exc_pop)
nest.Connect(inh_voltmeter, inh_pop)

## multimeter
#multimeter = nest.Create("multimeter")
#multimeter.set(record_from=["V_m"])

## spike recorder
spikerecorder = nest.Create("spike_recorder")

## weight recorder
#exc_wr = nest.Create('weight_recorder')
#nest.CopyModel(exc_synapse_model, "exc_stdp_synapse_rec", {"weight_recorder": exc_wr})
#inh_wr = nest.Create('weight_recorder')
#nest.CopyModel(inh_synapse_model, "inh_stdp_synapse_rec", {"weight_recorder": inh_wr})

## connections to devices
#nest.Connect(multimeter, exc_pop)
#nest.Connect(multimeter, inh_pop)
nest.Connect(exc_pop, spikerecorder)
nest.Connect(inh_pop, spikerecorder)

In [ ]:
exc_inh_w_per_iter = []
for _ in range(iterations):
    nest.Simulate(simtime)
    exc_inh_conn = nest.GetConnections(exc_pop, inh_pop)
    #print(exc_inh_w)
    exc_inh_w_per_iter.append(exc_inh_conn.get("weight"))

In [ ]:
# Extracting voltimeter data
def get_neuron_vm(neuron_idx):
    if neuron_idx >= exc_n+inh_n:
        raise Exception(f"neuron idx should be < {exc_n+inh_n}")
    if neuron_idx >= exc_n:
        print("this is an inh neuron")
        dmm = exc_voltmeter.get()
    else:
        print("this is an exc neuron")
        dmm = inh_voltmeter.get()
    #dmm = multimeter.get()
    Vms = dmm["events"]["V_m"][neuron_idx::exc_n+inh_n]
    ts = dmm["events"]["times"][neuron_idx::exc_n+inh_n]
    return Vms, ts

neuron_idx = 200
Vms, ts = get_neuron_vm(neuron_idx)
plt.plot(ts, Vms)

In [ ]:
# extracting spikes data
dSD = spikerecorder.get("events")
evs = dSD["senders"]
ts = dSD["times"]
plt.plot(ts, evs, ".")

In [ ]:
# extracting exc weights value data
# TODO: Fix this function! is totally wrong
def get_weight_value(pre_neuron_idx, post_neuron_idx):
    exc_inh_w  = exc_inh_wr.get("events")
    print(exc_inh_w )
    w = exc_inh_w ["weights"][pre_neuron_idx::(inh_n+1)]
    ts = exc_inh_w["times"][pre_neuron_idx::(inh_n+1)]

    return ts, w

pre_idx = 0
post_idx = 0
ts, w = get_weight_value(pre_idx, post_idx)
plt.plot(ts, w, ".-")

In [ ]:
# extracting inh weights value data
# TODO: Fix this function! is totally wrong
def get_weight_value(pre_neuron_idx, post_neuron_idx):
    inh_exc_w = inh_exc_wr.get("events")
    print(inh_exc_w)
    w = inh_exc_w["weights"][post_neuron_idx::exc_n]
    ts = inh_exc_w["times"][post_neuron_idx::exc_n]

    return ts, w

pre_idx = 0
post_idx = 3
ts, w = get_weight_value(pre_idx, post_idx)
plt.plot(ts, w, ".-")

In [ ]:
print(exc_inh_w_per_iter)
time = np.arange(simtime, simtime*(iterations+1), simtime)
print(time)
plt.plot(time, exc_inh_w_per_iter)

In [ ]:
exc_noise

In [ ]:
#plt.plot(exc_noise)

In [ ]:
nest.GetDefaults("stdp0_nestml__with_edlif_psc_alpha_percent0_nestml")

In [ ]:
nest.GetDefaults("stdp_synapse")

In [ ]:
nest.GetDefaults("iaf_psc_alpha")

In [16]:
import numpy
test = nest.Create("edlif_psc_exp_percent", 5)
print(test)
#test.set({"ATP": nest.random.normal(mean=5, std=1)})
mean = 5
std = 1
test.set({"ATP": [mean+std*numpy.random.rand() for x in range(len(test))]})
test.get("ATP")

NodeCollection(metadata=None, model=edlif_psc_exp_percent, size=5, first=70, last=74)


(5.875132986818338,
 5.065239162160869,
 5.211942506448816,
 5.857455358310189,
 5.319984112431898)